In [28]:
sentence = "Hello world. Nice to meet you. I want to become your friend. Hello again."

In [29]:
alignment = {
    "characters": [
        "H", "e", "l", "l", "o", " ", "w", "o", "r", "l", "d", ".", " ", "N", "i", "c", "e", " ", "t", "o", " ",
        "m", "e", "e", "t", " ", "y", "o", "u", ".", " ", "I", " ", "w", "a", "n", "t", " ", "t", "o", " ", "b",
        "e", "c", "o", "m", "e", " ", "y", "o", "u", "r", " ", "f", "r", "i", "e", "n", "d", ".", " ", "H", "e",
        "l", "l", "o", " ", "a", "g", "a", "i", "n", "."
    ],
    "characterStartTimesSeconds": [
        0, 0.163, 0.244, 0.279, 0.337, 0.43, 0.522, 0.569, 0.662, 0.743, 0.836, 0.987, 1.057, 1.207, 1.3, 1.393, 1.451,
        1.498, 1.544, 1.567, 1.602, 1.66, 1.695, 1.753, 1.8, 1.846, 1.904, 1.939, 1.974, 2.09, 2.241, 2.415, 2.473, 2.566,
        2.601, 2.67, 2.728, 2.763, 2.81, 2.833, 2.868, 2.926, 2.961, 3.019, 3.088, 3.146, 3.193, 3.228, 3.262, 3.286, 3.309,
        3.355, 3.39, 3.448, 3.483, 3.576, 3.646, 3.68, 3.738, 3.889, 3.982, 4.156, 4.214, 4.296, 4.331, 4.389, 4.47, 4.551,
        4.609, 4.679, 4.772, 4.83, 4.981
    ],
    "characterEndTimesSeconds": [
        0.163, 0.244, 0.279, 0.337, 0.43, 0.522, 0.569, 0.662, 0.743, 0.836, 0.987, 1.057, 1.207, 1.3, 1.393, 1.451, 1.498,
        1.544, 1.567, 1.602, 1.66, 1.695, 1.753, 1.8, 1.846, 1.904, 1.939, 1.974, 2.09, 2.241, 2.415, 2.473, 2.566, 2.601, 2.67,
        2.728, 2.763, 2.81, 2.833, 2.868, 2.926, 2.961, 3.019, 3.088, 3.146, 3.193, 3.228, 3.262, 3.286, 3.309, 3.355, 3.39,
        3.448, 3.483, 3.576, 3.646, 3.68, 3.738, 3.889, 3.982, 4.156, 4.214, 4.296, 4.331, 4.389, 4.47, 4.551, 4.609, 4.679, 4.772, 4.83, 4.981, 5.294
    ]
}

In [30]:
len(alignment["characterStartTimesSeconds"])

73

In [58]:
import pronouncing
import re

def get_word_time_list(text, alignment):
    word_time_list = [] 
    viseme_sequence = []
    for match in re.finditer(r"\b\w+\b", text):
        word = match.group(0).lower()
        phones = pronouncing.phones_for_word(word)[0].split(" ")
        phones = [re.sub(r"\d+", "", x) for x in phones]
        visemes = [PHONEME_TO_VISEME[x] for x in phones]
        word_start_idx = match.start()

        prev_viseme = None
        for i, viseme in enumerate(visemes):
            if viseme == prev_viseme:
                continue
            start_time = alignment["characterStartTimesSeconds"][word_start_idx + i]
            viseme_sequence.append((viseme, int(start_time * 1000)))
            prev_viseme = viseme
        word_end_time = alignment["characterEndTimesSeconds"][word_start_idx + len(word) - 1]
        viseme_sequence.append((0, int(word_end_time * 1000)))
    return viseme_sequence

In [59]:
sentence

'Hello world. Nice to meet you. I want to become your friend. Hello again.'

In [60]:
get_word_time_list(sentence, alignment)

[(1, 0),
 (5, 244),
 (7, 279),
 (0, 430),
 (11, 522),
 (7, 569),
 (5, 662),
 (2, 743),
 (0, 987),
 (2, 1207),
 (1, 1300),
 (9, 1393),
 (0, 1498),
 (2, 1544),
 (11, 1567),
 (0, 1602),
 (6, 1660),
 (3, 1695),
 (2, 1753),
 (0, 1846),
 (3, 1904),
 (11, 1939),
 (0, 2090),
 (1, 2415),
 (0, 2473),
 (11, 2566),
 (1, 2601),
 (2, 2670),
 (0, 2763),
 (2, 2810),
 (11, 2833),
 (0, 2868),
 (6, 2926),
 (3, 2961),
 (2, 3019),
 (1, 3088),
 (6, 3146),
 (0, 3228),
 (3, 3262),
 (1, 3286),
 (8, 3309),
 (0, 3390),
 (4, 3448),
 (8, 3483),
 (3, 3576),
 (2, 3646),
 (0, 3889),
 (1, 4156),
 (5, 4296),
 (7, 4331),
 (0, 4470),
 (1, 4551),
 (2, 4609),
 (3, 4679),
 (2, 4772),
 (0, 4981)]

In [11]:
!uv add pronouncing

Resolved 109 packages in 0.41ms
Audited 105 packages in 0.02ms


In [45]:
import pronouncing
import re

def sentence_to_phonemes(sentence):
    # Simple tokenization, remove punctuation, convert to lowercase
    words = re.findall(r"\b\w+\b", sentence.lower())
    phoneme_dict = {} 
    for word in words:
        phones = pronouncing.phones_for_word(word)
        if phones:
            # Use the first pronunciation if multiple exist
            phonemes = phones[0].split()
            phoneme_dict[word] = phonemes
        else:
            phoneme_dict[word] = ['<UNK>']  # Unknown word
    return phoneme_dict

In [46]:
sentence_to_phonemes(sentence)

{'hello': ['HH', 'AH0', 'L', 'OW1'],
 'world': ['W', 'ER1', 'L', 'D'],
 'nice': ['N', 'AY1', 'S'],
 'to': ['T', 'UW1'],
 'meet': ['M', 'IY1', 'T'],
 'you': ['Y', 'UW1'],
 'i': ['AY1'],
 'want': ['W', 'AA1', 'N', 'T'],
 'become': ['B', 'IH0', 'K', 'AH1', 'M'],
 'your': ['Y', 'AO1', 'R'],
 'friend': ['F', 'R', 'EH1', 'N', 'D'],
 'again': ['AH0', 'G', 'EH1', 'N']}

SyntaxError: expected ':' (1386205534.py, line 1)

In [44]:
PHONEME_TO_VISEME = {
    "AA": 1, "AE": 1, "AH": 1, "AO": 1, "AW": 1, "AY": 1, "HH": 1, 
    "D": 2,  "G": 2,  "K": 2,  "N": 2,  "T": 2,  "NG": 2,
    "Y": 3,  "EY": 3, "EH": 3, "IH": 3,  "IY": 3,
    "F": 4,  "V": 4, 
    "L": 5, 
    "B": 6, "M": 6,  "P": 6, 
    "ER": 7, "OW": 7,  "OY": 7,
    "R": 8, 
    "S": 9,  "Z": 9, "CH": 9,  "DH": 9,  "JH": 9,  "SH": 9, "TH": 9,  "ZH": 9,
    "UH": 10, 
    "UW": 11, "W": 11
}

In [2]:
!uv add elevenlabs

Resolved 115 packages in 325ms                                       
Installed 6 packages in 45ms0                                    
 + annotated-types==0.7.0
 + elevenlabs==2.8.0
 + pydantic==2.11.7
 + pydantic-core==2.33.2
 + typing-inspection==0.4.1
 + websockets==15.0.1


In [3]:
from elevenlabs import ElevenLabs

In [8]:
import time

client = ElevenLabs(
    api_key="sk_423c17be6c3ebd0b16a7ae2d3d2a79ca929d635d17de40fd",
)
response = client.text_to_speech.stream_with_timestamps(
    voice_id="JBFqnCBsd6RMkjVDRZzb",
    output_format="mp3_44100_128",
    text="The first move is what sets everything in motion.",
    model_id="eleven_multilingual_v2",
)

start = time.time()
for i, chunk in enumerate(response):
    elapsed = time.time() - start
    print(i, elapsed, type(chunk))

0 1.8750290870666504 <class 'elevenlabs.types.streaming_audio_chunk_with_timestamps_response.StreamingAudioChunkWithTimestampsResponse'>
1 1.8763248920440674 <class 'elevenlabs.types.streaming_audio_chunk_with_timestamps_response.StreamingAudioChunkWithTimestampsResponse'>
2 1.8769071102142334 <class 'elevenlabs.types.streaming_audio_chunk_with_timestamps_response.StreamingAudioChunkWithTimestampsResponse'>
3 1.8822600841522217 <class 'elevenlabs.types.streaming_audio_chunk_with_timestamps_response.StreamingAudioChunkWithTimestampsResponse'>
4 1.8976490497589111 <class 'elevenlabs.types.streaming_audio_chunk_with_timestamps_response.StreamingAudioChunkWithTimestampsResponse'>
5 1.906360149383545 <class 'elevenlabs.types.streaming_audio_chunk_with_timestamps_response.StreamingAudioChunkWithTimestampsResponse'>
6 1.910189151763916 <class 'elevenlabs.types.streaming_audio_chunk_with_timestamps_response.StreamingAudioChunkWithTimestampsResponse'>
7 1.916694164276123 <class 'elevenlabs.type